In [1]:
import requests as rq
import json
import pandas as pd
pd.set_option('display.precision', 4,
              'display.colheader_justify', 'center')
import numpy as np
import warnings
import pytz
import datetime
import time
from IPython.display import clear_output

In [2]:
# Get pro API key
def get_pro_key():
    f = open("/home/vikas/Documents/CG_pro_key.json")
    key_dict = json.load(f)
    return key_dict["key"]

In [3]:
use_pro = {
         "accept": "application/json",
         "x-cg-pro-api-key" : get_pro_key()
}

In [43]:
def get_response(endpoint, headers, params, URL):
    url = "".join((URL, endpoint))
    response = rq.get(url, headers = headers, params = params)
    if response.status_code == 200:
        data = response.json()
        return data
    else:
        print(f"Failed to fetch data, check status code {response.status_code}")

In [5]:
PUB_URL = "https://api.coingecko.com/api/v3"
PRO_URL = "https://pro-api.coingecko.com/api/v3"

## Get list of networks

In [6]:
networks_list_response = get_response("/onchain/networks",
                                      use_pro, 
                                      "",
                                      PRO_URL)

In [7]:
networks_list_response["data"][0]

{'id': 'eth',
 'type': 'network',
 'attributes': {'name': 'Ethereum', 'coingecko_asset_platform_id': 'ethereum'}}

In [8]:
df_net = pd.DataFrame(networks_list_response["data"])

## Get DEX for a specific network

In [9]:
network = "eth"

dex_list_response = get_response(f"/onchain/networks/{network}/dexes",
                                 use_pro, 
                                 "",
                                 PRO_URL)

In [10]:
df_dex = pd.DataFrame(dex_list_response["data"])

## Create URLs

In [70]:
def get_url(url_type, network, dex = "", pool_address = ""):   

    url_dict = {
        "trending_pools": f"/onchain/networks/{network}/trending_pools",
        "top_pools": f"/onchain/networks/{network}/pools",
        "top_pools_dex": f"/onchain/networks/{network}/dexes/{dex}/pools",
        "specific_pool_dex": f"/onchain/networks/{network}/pools/{pool_address}"
    }

    return url_dict[url_type]

## Get trending pools for a specific network

In [12]:
def collect_response(list_response):    

    response_all = []

    for response in list_response["data"]:
        all_attributes = response["attributes"]
        daily_tx = all_attributes["transactions"]["h24"]
        rel = response["relationships"]
        
        temp_dict = dict(
            pair = all_attributes["name"],
            dex = rel["dex"]["data"]["id"],
            add = all_attributes["address"],
            fdv_usd = all_attributes["fdv_usd"],
            market_cap_usd = all_attributes["market_cap_usd"],
            daily_volume = all_attributes["volume_usd"]["h24"],
            daily_price_change = all_attributes["price_change_percentage"]["h24"],
            daily_buys = daily_tx["buys"],
            daily_sells = daily_tx["sells"],
            daily_buyers = daily_tx["buyers"],
            daily_sellers = daily_tx["sellers"]
        )
        
        response_all.append(temp_dict)

    return response_all

In [68]:
def get_trending_pools(network, sort_by_col):

    target_url = get_url("trending_pools", network)

    trendpool_list_response = get_response(target_url,
                                           use_pro, 
                                           "",
                                           PRO_URL)
    
    trendpool_all = collect_response(trendpool_list_response)    

    return pd.DataFrame(trendpool_all).sort_values(by = [f"{sort_by_col}"],
                                                   ascending = False)

In [71]:
#get_trending_pools("eth", "daily_volume")

## Get top pools for a specific network

In [72]:
def get_top_pools_network(network, sort_by_col):

    target_url = get_url("top_pools", network)

    toppool_list_response = get_response(target_url,
                                         use_pro, 
                                         "",
                                         PRO_URL)

    toppool_all = collect_response(toppool_list_response)   

    return pd.DataFrame(toppool_all).sort_values(by = [f"{sort_by_col}"],
                                                 ascending = False)

In [74]:
#get_top_pools_network("eth", "daily_volume")

## Get top pools for a specific DEX

In [75]:
def get_top_pools_dex(network, dex, sort_by_col):

    target_url = get_url("top_pools_dex", network, dex)

    toppool_list_response = get_response(target_url,
                                         use_pro, 
                                         "",
                                         PRO_URL)

    toppool_all = collect_response(toppool_list_response)   

    return pd.DataFrame(toppool_all).sort_values(by = [f"{sort_by_col}"],
                                                 ascending = False)

In [77]:
get_top_pools_dex("eth", "uniswap_v2", "market_cap_usd")

,pair,dex,add,fdv_usd,market_cap_usd,daily_volume,daily_price_change,daily_buys,daily_sells,daily_buyers,daily_sellers
4,DOGGO / WETH,uniswap_v2,0x7eb6d3466600b4857eb60a19e0d2115e65aa815e,7328362,7725268.10683813,8785841.06971933,-46.17,2748,2052,1395,1142
14,WETH / USDT,uniswap_v2,0x0d4a11d5eeaac28ec3f61d100daf4d40471f1852,7602459394,7583985845.88759,5075980.91272712,0.01,1775,1007,1140,785
0,MARS / WETH,uniswap_v2,0x284fedfb2c5b90877d42b584a790b824dc869fad,9212186,None,10165602.9889907,-12.71,3333,2612,1891,1456
1,INCEPT / WETH,uniswap_v2,0x204c6acf0bda169581c237fdbc02827b1ce6f7aa,9878775,None,12201899.4559431,98768.42,3595,2207,1932,1105
2,ESTEE / WETH,uniswap_v2,0xe2c456b849be4cd14d8422d9868953ac60633e0d,512306,None,6584372.44682,-54.58,3142,2608,1445,1245
3,HANA / WETH,uniswap_v2,0xe7b4e528308c84fd6698906b6224615e9e30d236,2173941,None,7836984.6880472,-55.15,3268,2452,1537,1300
5,KAGASOU / WETH,uniswap_v2,0x1408743cdc399b0d00e132f841ad1a6e11bba3a0,190625,None,2899203.38834143,8283.32,2688,2093,1452,1107
6,KAGASOU / WETH,uniswap_v2,0x8e5f8836208badd3280b5d9b7e908deb65bea806,34753.69,None,2184334.09877461,848.91,2143,1714,1336,1019
7,PESTO / WETH,uniswap_v2,0xbc6a213cbb34424670548ef4b3388f5fdbd07992,1224636,None,3618183.93925336,-37.21,2074,1578,1116,941
8,OSCAR / WETH,uniswap_v2,0x9b9d97a9215882f4a68b0d0d591d5aaeb29e5e47,109101,None,3711453.23575709,3123.68,2110,1513,1183,824


## Get data for a specific pool address

In [53]:
def collect_pool_response(list_response):    

    response = list_response["data"]
    all_attributes = response["attributes"]
    daily_tx = all_attributes["transactions"]["h24"]
    rel = response["relationships"]
        
    response_dict = dict(
        pair = all_attributes["name"],
        dex = rel["dex"]["data"]["id"],
        add = all_attributes["address"],
        fdv_usd = all_attributes["fdv_usd"],
        market_cap_usd = all_attributes["market_cap_usd"],
        daily_volume = all_attributes["volume_usd"]["h24"],
        daily_price_change = all_attributes["price_change_percentage"]["h24"],
        daily_buys = daily_tx["buys"],
        daily_sells = daily_tx["sells"],
        daily_buyers = daily_tx["buyers"],
        daily_sellers = daily_tx["sellers"]
    )

    return response_dict

In [78]:
def get_pool_data(network, dex, pool_address):

    target_url = get_url("specific_pool_dex", network, dex, pool_address)

    pool_list_response = get_response(target_url,
                                     use_pro, 
                                     "",
                                     PRO_URL)

    pool_all = collect_pool_response(pool_list_response)   

    return pool_all

In [79]:
get_pool_data("eth", "uniswap_v2", "0x0d4a11d5eeaac28ec3f61d100daf4d40471f1852")

{'pair': 'WETH / USDT',
 'dex': 'uniswap_v2',
 'add': '0x0d4a11d5eeaac28ec3f61d100daf4d40471f1852',
 'fdv_usd': '7602760815',
 'market_cap_usd': '7583985845.88759',
 'daily_volume': '5078808.81990872',
 'daily_price_change': '0.01',
 'daily_buys': 1776,
 'daily_sells': 1010,
 'daily_buyers': 1140,
 'daily_sellers': 787}